In [1]:
import json
import os
import pandas as pd
import load_geolife
import add_geohash
import create_nodes
import geohash_location_count
import geohash_user_count
import create_geohash_meet
import create_geohash_intervals
import geohash_pairs
import find_duplicates

In [9]:
def create_dirs():
    if not os.path.isdir('images/'):
        os.makedirs('images/')
    if not os.path.isdir('htmls/'):
        os.makedirs('htmls/')
    if not os.path.isdir('data/'):
        os.makedirs('data/')
    if not os.path.isdir('data/duplicates'):
        os.makedirs('data/duplicates')
    if not os.path.isdir('data/geolife_geohash_intervals'):
        os.makedirs('data/geolife_geohash_intervals')
    if not os.path.isdir('data/geolife_pairs'):
        os.makedirs('data/geolife_pairs')


def removeDuplicates(in_file, duplicates_file, out_file):
    duplicates = pd.read_csv(duplicates_file)
    df = pd.read_csv(in_file)
    df['px'] = df['Person ID'].astype('string') + "_" + df['Trajectory'].astype('string') + ".plt"
    duplicates['px'] = duplicates['user'].astype('string')+"_"+duplicates['file_name'].astype('string')
    noduplicates = df[~df['px'].isin(duplicates['px'])].copy(deep=True)
    noduplicates.to_csv(out_file,index=False)

In [3]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

In [10]:
create_dirs()
# load_geolife.load_geolife('./geolife/Data/',"./data/geolife_gps_data.csv", "./data/geolife_labels_data.csv")
# add_geohash.add_geohash('./data/geolife_gps_data.csv', './data/geolife_geohash_size_8.csv')
# find_duplicates.find_duplicates_by_filename('./geolife/Data/', './data/duplicates/')
# find_duplicates.find_duplicates_by_content('./geolife/Data/', './data/duplicates/')
# removeDuplicates('./data/geolife_geohash_size_8.csv', './data/duplicates/geolife_user_track_duplicates_by_content.csv', './data/geolife_geohash_size_8_no_duplicates.csv')
# create_nodes.create_nodes('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_nodes.csv')
# geohash_location_count.geohash_location_count('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_geohash_8_location_counts.csv')
# geohash_user_count.geohash_user_count('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_geohash_8_user_counts.csv')
# count_files = create_geohash_meet.createGeohashMeetJsons('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_geohash_meet/')
# create_geohash_meet.createGeohashMeetCSV(count_files,'./data/geolife_geohash_meet/', './data/geolife_geohash_meet_size_8.csv')
# create_geohash_intervals.create_geohash_intervals('./data/geolife_geohash_meet_size_8.csv', './data/geolife_geohash_meet_intervals_size_8.csv')
# geohash_pairs.transform_times('./data/geolife_geohash_meet_intervals_size_8.csv','./data/geolife_geohash_intervals/','./data/geolife_geohash_8_intervals_table.csv')
# geohash_pairs.split_to_single_day('./data/geolife_geohash_8_intervals_table.csv','./data/geolife_geohash_8_intervals_single_day.csv')
# geohash_pairs.create_date_pairs('./data/geolife_geohash_8_intervals_single_day.csv','./data/geolife_geohash_8_day_meets.csv')
geohash_pairs.create_pairs('./data/geolife_geohash_8_intervals_single_day.csv','./data/geolife_pairs/', './data/geolife_geohash_8_pairs.csv')
geohash_pairs.create_edges('./data/geolife_geohash_8_pairs.csv', './data/geolife_geohash_size_8_edges.csv')

100%|██████████| 5046480/5046480 [1:08:02<00:00, 1236.26it/s]


Local

In [4]:
import pandas as pd
import json
from tqdm import tqdm

def create_geohash_time_meets_local():
    df = pd.read_csv('data/geolife_geohash_meet_intervals_size_8.csv')
    meets_dict = {}

    for index, row in tqdm(df.iterrows()):
        meets_intervals = json.loads(row['meets_intervals'])
        
        if len(meets_intervals) > 1:
            keys = list(meets_intervals.keys())
            intervals = [meets_intervals[key] for key in keys]
            
            for i in range(len(keys)):
                for j in range(i + 1, len(keys)): 
                    for interval_i in intervals[i]:
                        for interval_j in intervals[j]:
                            if (interval_i[1] <= interval_j[2] and interval_j[1] <= interval_i[2]):
                                key = tuple(sorted([keys[i], keys[j]]))
                                # Add the meeting information to the dictionary, including the geohash and timestamp
                                meets_dict.setdefault((row['geohash'], interval_i[5]), set()).add(key)

        else:
            key = next(iter(meets_intervals))
            # Add the meeting information to the dictionary, including the geohash and timestamp
            meets_dict.setdefault((row['geohash'], meets_intervals[key][0][5]), set()).add(key)

    # Flatten the dictionary to create the DataFrame
    new_data = [{'geohash': key[0], 'time': key[1], 'person_ids': list(person_pairs)} for key, person_pairs in meets_dict.items()]
    new_df = pd.DataFrame(new_data)

    new_df.to_csv('data2/geolife_local_meets.csv', index=False)

create_geohash_time_meets_local()


415946it [00:43, 9624.37it/s] 


Global

In [5]:
import pandas as pd
import json
from tqdm import tqdm

def create_geohash_time_meets_global():
    df = pd.read_csv('data/geolife_geohash_meet_intervals_size_8.csv')
    meets_dict = {}

    for index, row in tqdm(df.iterrows()):
        meets_intervals = json.loads(row['meets_intervals'])
        
        if len(meets_intervals) > 1:
            keys = list(meets_intervals.keys())
            intervals = [meets_intervals[key] for key in keys]
            
            for i in range(len(keys)):
                for j in range(i + 1, len(keys)): 
                    for interval_i in intervals[i]:
                        for interval_j in intervals[j]:
                            if (interval_i[1] <= interval_j[2] and interval_j[1] <= interval_i[2]):
                                key = tuple(sorted([keys[i], keys[j]]))
                                # Use the start time of interval_i for timestamp
                                timestamp = interval_i[5]  # Assuming start time is at index 5
                                meets_dict.setdefault(key, {'users': key, 'meetings': [], 'geohashes': set(), 'time': [None, None]})
                                meets_dict[key]['meetings'].append((row['Latitude'], row['Longitude']))
                                meets_dict[key]['geohashes'].add(row['geohash'])
                                if meets_dict[key]['time'][0] is None or interval_i[1] < meets_dict[key]['time'][0]:
                                    meets_dict[key]['time'][0] = interval_i[1]
                                if meets_dict[key]['time'][1] is None or interval_j[2] > meets_dict[key]['time'][1]:
                                    meets_dict[key]['time'][1] = interval_j[2]

    new_data = [{'users': info['users'], 'meetings': info['meetings'], 'geohashes': list(info['geohashes']), 'time': info['time']} for info in meets_dict.values()]
    new_df = pd.DataFrame(new_data)

    new_df.to_csv('data2/geolife_global_meets.csv', index=False)

create_geohash_time_meets_global()


415946it [00:44, 9266.63it/s] 
